In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter

In [2]:
from sklearn.model_selection import KFold

In [3]:
class KNN:
    #constructor
    def __init__(self,neighbours=5):
        self.neighbours = neighbours
        
    def fit(self,X,y):
        self.X = np.array(X)
        self.y = np.array(y)
          
    def predict_point(self,point):
        point = np.array(point)
        all_dist = []
        for x_point,y_point in zip(self.X,self.y):
            dist = ((x_point-point)**2).sum()
            all_dist.append([dist,y_point])
        all_dist.sort()
        all_dist = all_dist[:self.neighbours]
        k_nearest_labels = [label for distance,label in all_dist]
        return Counter(k_nearest_labels).most_common()[0][0]
     
        
    def predict(self,all_points):
        result = []
        for point in all_points:
            result.append(self.predict_point(point))
        return result
    
    def score(self,X_test,y_test):
        result = np.array(self.predict(X_test))
        y = np.array(y_test)
        return sum(result==y)/len(y)
        
        

In [4]:
class KnnWithBestK:
    def __init__(self,start,end):
        if start%2==0:
            self.start = start+1
        else:
            self.start = start
        self.end = end
        
    def fit(self,X,y,k=5):
        kf = KFold(n_splits=k,shuffle = True)
        k_value = range(self.start,self.end+1)
        accuracy = []
        for K in k_value:
            kfold_accuracy = []
            for i in range(k):
                result = next(kf.split(X),None)
                X_train = X[result[0]]
                X_val = X[result[1]]
                y_train = y[result[0]]
                y_val = y[result[1]]
                model = KNN(K)
                model.fit(X_train,y_train)
                kfold_accuracy.append(model.score(X_val,y_val))
            accuracy.append(np.mean(kfold_accuracy))
        self.neighbours = k_value[np.argmax(accuracy)]
        self.X = X
        self.y = y
    
    def predict(self,X_test):
        model = KNN(self.neighbours)
        model.fit(self.X,self.y)
        result = model.predict(X_test)
        return result
    def score(self,X_test,y_test):
        model = KNN(self.neighbours)
        model.fit(self.X,self.y)
        return model.score(X_test,y_test)

In [5]:
from sklearn import datasets

In [6]:
iris_data = datasets.load_iris()

In [7]:
X = iris_data.data
y = iris_data.target

In [8]:
from sklearn.preprocessing import normalize

In [76]:
X = normalize(X,axis=0)

In [77]:
from sklearn.model_selection import train_test_split

In [78]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.23,shuffle=True,random_state = 42)

In [79]:
X_train.shape

(115, 4)

In [80]:
X_test.shape

(35, 4)

In [81]:
model = KNN()
model.fit(X_train,y_train)

In [85]:
y_pred = np.array(model.predict(X_test))

In [86]:
sum(y_pred==y_test)

35

In [87]:
len(y_test)

35

In [95]:
model2 = KnnWithBestK(1,10)

In [96]:
model2.fit(X_train,y_train)

In [97]:
model2.score(X_test,y_test)

1.0

In [98]:
from sklearn.neighbors import KNeighborsClassifier

In [99]:
model = KNeighborsClassifier()